In [16]:
import math

class Vector:
    def __init__(self, coordinate):
        self.coordinate = coordinate
    def __str__(self):
        return 'Vector:{}'.format(self.coordinate)
    def __add__(self, other):
        return Vector([x+y for x,y in zip(self.coordinate, other.coordinate)])
    def __sub__(self, other):
        return Vector([x-y for x,y in zip(self.coordinate, other.coordinate)])
    def __mul__(self, value):
        return Vector([x*value for x in self.coordinate])
    def magnitude(self):
        return math.sqrt(sum([x**2 for x in self.coordinate]))
    def normalized(self):
        magnitude = self.magnitude()
        return Vector([x/magnitude for x in self.coordinate])
    def dot(self, other):
        return sum([x*y for x,y in zip(self.coordinate, other.coordinate)])  
    def angle(self, other):
        dot_products = self.dot(other)
        mag_self = self.magnitude()
        mag_other = other.magnitude()
        result = math.acos(dot_products/(mag_self*mag_other))
        return result   
    def angle_degree(self, other):
        return math.degrees(self.angle(other))
    def is_zero(self, tolerance=1e-10):
        return self.magnitude()<tolerance
    def is_parallel_to(self, other):
        return (self.is_zero() or 
               other.is_zero() or
               self.angle(other) == 0 or
               self.angle(other) == math.pi)
    def parallel_to(self, basis):
        u = basis.normalized()
        weight = self.dot(u)
        return u*weight
    def orthogonal_to(self, basis):
        projection = self.parallel_to(basis)
        return self-projection
    def __iter__(self):
        self.count = 0
        return self
    
    def __next__(self):
        self.count += 1
        if self.count>len(self.coordinate):
            raise StopIteration
        return self.coordinate[self.count-1]
    
    def __getitem__(self,index):
        return self.coordinate[index]
    def __setitem__(self,index,value):
        self.coordinate[index] = value
    

In [19]:
from decimal import Decimal, getcontext

getcontext().prec = 30


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = Decimal(n[initial_index])

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [20]:
l1 = Line(Vector([4.046, 2.836]), 1.21)
print(l1)

4.046x_1 + 2.836x_2 = 1.210


In [29]:
v_1 = Vector([-0.221, 7.437])
v_1.magnitude()

7.440282924728065

In [30]:
v_2 = Vector([8.813,-1.331,-6.247])
v_2.magnitude()

10.884187567292289

In [31]:
v_3 = Vector([5.581, -2.136])
print(v_3.normalized())

Vector:[0.9339352140866403, -0.35744232526233]


In [32]:
v_4 = Vector([1.996, 3.108, -4.554])
print(v_4.normalized())

Vector:[0.3404012959433014, 0.5300437012984873, -0.7766470449528028]


In [36]:
v_5 = Vector([7.887,4.138])
v_6 = Vector([-8.802,6.776])
print(v_5.dot(v_6))

-41.382286


In [37]:
v_1 = Vector([-5.955, -4.904, -1.874])
v_2 = Vector([-4.496, -8.755, 7.103])
print(v_1.dot(v_2))

56.397178000000004


In [39]:
v_1 = Vector([3.183, -7.627])
v_2 = Vector([-2.668, 5.319])
print(v_1.angle(v_2))

3.0720263098372476


In [42]:
v_1 = Vector([7.35,0.221,5.188])
v_2 = Vector([2.751,8.259,3.985])
print(v_1.angle_degree(v_2))

60.27581120523091


In [50]:
v_1 = Vector([-7.579, -7.88])
v_2 = Vector([22.737, 23.64])
print(v_1.is_parallel_to(v_2))
print(v_1.dot(v_2))

True
-358.60692299999994


In [54]:
v = Vector([3.039, 1.879])
w = Vector([0.825, 2.036])
print(v.parallel_to(w))

Vector:[1.0826069624844668, 2.671742758325302]


In [53]:
v = Vector([-9.88, -3.264, -8.159])
w = Vector([-2.155, -9.353, -9.473])
print(v.orthogonal_to(w))

Vector:[-8.350081043195763, 3.376061254287722, -1.4337460427811841]


In [56]:
v = Vector([3.009, -6.172, 3.692, -2.51 ])
b = Vector([6.404, -9.144, 2.759, 8.718])
print(v.parallel_to(b))
print(v.orthogonal_to(b))

Vector:[1.9685161672140898, -2.8107607484393564, 0.8480849633578503, 2.679813233256158]
Vector:[1.04048383278591, -3.3612392515606433, 2.8439150366421497, -5.189813233256158]
